### Import the required packages into python

In [ ]:
import pulp
import itertools
import gurobipy

### Set up the Test C problem

In [ ]:
def initialise_testc_minimisation_problem(
specialties, hospitals, bands, regions, scenarios
):
    """
    Initialise the minimisation problem.
    Set decision variables for the models.
    
    """
    sh = [(s,h) for s in specialties for h in hospitals]
    shb = [(s,h,b) for s in specialties for h in hospitals for b in bands]
    shk = [(s,h,k) for s in specialties for h in hospitals for k in scenarios]
    srhk = [(s,r,h,k) for s in specialties for r in regions for h in hospitals for k in scenarios]
    sbhk = [(s,b,h,k) for s in specialties for b in bands for h in hospitals for k in scenarios]
    
    prob = pulp.LpProblem("Stochastic", pulp.LpMinimize)
    
    xbed = pulp.LpVariable.dicts(
        "Xbed", (specialties,hospitals), lowBound=0, cat = 'Integer'
    )
    xstaff = pulp.LpVariable.dicts(
        "Xstaff", (specialties,hospitals,bands), lowBound=0,cat = 'Integer'
    )
    ubed = pulp.LpVariable.dicts(
        "Ubed",(specialties,hospitals,scenarios), lowBound=0, cat='Integer'
    )
    ustaff = pulp.LpVariable.dicts(
        "Ustaff",(specialties,hospitals,bands,scenarios), lowBound=0, cat='Integer'
    )
    return prob, sh, shb, shk, srhk, sbhk, xbed, xstaff, ubed, ustaff

def add_testc_constraints(
    xbed, xstaff, ubed, ustaff, UBbed, UBstaff, UBubed, UBustaff, D, R, K, prob, sh, shb, shk, srhk, sbhk, First_stage
):
    
    """
    Add the constraints that are required for the Test C model
    
    - Constraints 1-6: Ensures demand is met across all specialties and all regions
    - Constraint 7: Ensures beds are only able to open in a ward if the facilities are able to be opened - 1st stage
    - Constraint 8: Ensures beds are only able to open in a ward if the facilities are able to be opened - 2nd stage
    - Constraint 9: Ensures staffing ratios are met in the first stage
    - Constraint 10: Ensures staffing ratios are met in the second stage
    - Constraint 11: Ensures beds deployed does not exceed maximum capacity of hospital - 1st stage
    - Constraint 12: Ensures beds deployed does not exceed maximum capacity of hospital - 2nd stage
    - Constraint 13: Ensures staff deployed does not exceed maximum staffing resources - 1st stage
    - Constraint 14: Ensures staff deployed does not exceed maximum staffing resources - 2nd stage
    - Constraint 15: Deterministic values must be met as a minimum for the first stage
      """
        
    for k in scenarios:
        for s in specialties:
            prob += pulp.lpSum(xbed[s][h] + ubed[s][h][k] for h in region1) >= pulp.lpSum(D[s][0][k]) #Constraint 1
            prob += pulp.lpSum(xbed[s][h] + ubed[s][h][k] for h in region2) >= pulp.lpSum(D[s][1][k]) #Constraint 2
            prob += pulp.lpSum(xbed[s][h] + ubed[s][h][k] for h in region3) >= pulp.lpSum(D[s][2][k]) #Constraint 3
            prob += pulp.lpSum(xbed[s][h] + ubed[s][h][k] for h in region4) >= pulp.lpSum(D[s][3][k]) #Constraint 4
            prob += pulp.lpSum(xbed[s][h] + ubed[s][h][k] for h in region5) >= pulp.lpSum(D[s][4][k]) #Constraint 5
            prob += pulp.lpSum(xbed[s][h] + ubed[s][h][k] for h in region6) >= pulp.lpSum(D[s][5][k]) #Constraint 6 
                  
    for s in specialties:
        for h in hospitals:
            prob += pulp.lpSum(xbed[s][h]) <= pulp.lpSum(K[s][h]) #Constraint 7
            
    for s in specialties: 
        for h in hospitals:
            prob += pulp.lpSum(ubed[s][h][k] for k in scenarios)<= pulp.lpSum(K[s][h]) #Constraint 8
            
            for b in bands:
                prob += pulp.lpSum(xstaff[s][h][b]) >= pulp.lpSum(R[s][b]*(xbed[s][h])) #Constraint 9
                
                for k in scenarios:
                    prob += pulp.lpSum(ustaff[s][h][b][k])>= pulp.lpSum(R[s][b]*(ubed[s][h][k])) #Constraint 10
                    
    for h in hospitals:
        prob += pulp.lpSum(xbed[s][h] for s in specialties) <= UBbed[h] #Constraint 11
        
    for k in scenarios: 
        for h in hospitals:
            prob += pulp.lpSum(ubed[s][h][k] for s in specialties) <=UBubed[h][k] #Constraint 12
        
    for b in bands:
        prob += pulp.lpSum(xstaff[s][h][b] for (s,h) in sh) <= UBstaff[b] #Constraint 13
        
        for k in scenarios:
            prob += pulp.lpSum(ustaff[s][h][b][k] for (s,h) in sh) <= UBustaff[b][k] #Constraint 14
    for s in specialties:
        for h in hospitals:
            prob += pulp.lpSum(xbed[s][h]) >= First_stage[s][h]         
    return prob
def solve_testc_minimisation_problem(
    specialties,
    bands,
    hospitals,
    regions,
    scenarios,
    pscenarios,
    D,
    R,
    K,
    c1bed,
    c2bed,
    c1staff,
    c2staff,
    UBbed,
    UBstaff,
    First_stage
):
    """
    Solves the Test C problem, with the objective function being minimised
    """
    prob, sh, shb, shk, srhk, sbhk, xbed, xstaff, ubed, ustaff = initialise_stochastic_minimisation_problem(
        specialties=specialties, 
        hospitals=hospitals,
        bands=bands, 
        regions=regions, 
        scenarios=scenarios
    )
    prob +=(
        pulp.lpSum((xbed[s][h]*c1bed[s][h]) for (s,h) in sh) +
        pulp.lpSum((xstaff[s][h][b]*c1staff[b]) for (s,h,b) in shb) + 
        pulp.lpSum(pscenarios[k]*(ubed[s][h][k]*c2bed[s][h]) for (s,h,k) in shk)+ 
        pulp.lpSum(pscenarios[k]*(ustaff[s][h][b][k]*c2staff[b]) for (s,b,h,k) in sbhk)
    )

    prob = add_testc_constraints(
        xbed=xbed, 
        xstaff=xstaff,
        ubed=ubed,
        ustaff=ustaff,
        UBbed=UBbed, 
        UBstaff=UBstaff,
        UBubed=UBubed,
        UBustaff=UBustaff,
        D=D, 
        R=R,
        K=K,
        sh=sh,
        shb=shb,
        shk=shk,
        srhk=srhk,
        sbhk=sbhk,
        prob=prob,
        First_stage=First_stage
    )
    # The user can select one of the two optimisers
    # prob.solve(pulp.GUROBI())
    # prob.solve(pulp.PULP_CBC_CMD())
    return prob


### The following cell can be edited to allow the user to enter their own parameters and values for the model

In [ ]:
specialties = list(itertools.chain(range(0, ))) #Creates list of specialties
bands = list(itertools.chain(range(0, ))) #Creates list of nursing bands
regions = list(itertools.chain(range(0, ))) #Creates List of regions

region1 = []
region2 = []
region3 = []
region4 = []
region5 = []
region6 = []
hospitals = region1 + region2 +region3 + region4 +region5 +region6
D = [
[[],[]],
]
K = [
[],
]
R = [
[],
] 
c1bed = [
[],
]
c2bed = [
[],
]
c1staff = []
c2staff = []
UBstaff = [
[],
]
UBustaff = [
[],
]
UBbed = [
[],
]
UBubed =[
[],
]
scenarios = []
pscenarios = []
First_stage= [
[],
]

### The following runs the Test C model

In [ ]:
prob = solve_testc_minimisation_problem(
    specialties,
    bands,
    hospitals,
    regions,
    scenarios,
    pscenarios,
    D,
    R,
    K,
    c1bed,
    c2bed,
    c1staff,
    c2staff,
    UBbed,
    UBstaff,
    First_stage
)


### The following outputs the results of the Test C model

In [ ]:
print("Solution Status = ", pulp.LpStatus[prob.status])
print("Total price = ", pulp.value(prob.objective))  
for v in prob.variables():
    if v.varValue >= 0:
        print(v.name, "=", v.varValue)
production = [v.varValue for v in prob.variables()]